In [ ]:
# !pip install louvain
# !pip install --upgrade --no-cache psutil gputil

In [1]:
import os
import subprocess
import sys
sys.path.append('..')
from tools.formating.formating import *
from tools.utils.datasplit import *
from tools.qc.scanpy_qc import *
import numpy as np
import scipy

R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: MatrixGenerics

R[write to console]: Loading required package: matrixStats

R[write to console]: 
Attaching package: ‘MatrixGenerics’


R[write to console]: The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowC


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Attaching SeuratObject

R[write to console]: Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode

R[write to console]: 
Attaching package: ‘Seurat’


R[write to console]: The following object is masked from ‘package:SummarizedExperiment’:

    Assays


R[write to console]: Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

R[write to console]: Signac built for for SeuratObject v4 was just loaded with SeuratObject
v5; disabling v5 assays and validation routines, and ensuring assays
work in strict v3/v4 compatibility mode



scanpy==1.9.5 anndata==0.9.2 umap==0.5.4 numpy==1.22.4 scipy==1.9.1 pandas==1.3.5 scikit-learn==1.3.1 statsmodels==0.14.0 igraph==0.10.8 louvain==0.8.1 pynndescent==0.5.10


In [2]:
# input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_germ line.h5ad"
# input = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad"
# input = "/ps/ai-ready/data/error/hca_heart_neuronal_raw.h5ad"
# input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin.h5ad"
# input = "/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
# input = "/ps/ai-ready/data/error/hca_heart_neuronal_raw.h5ad"
# input = "/ps/ai-ready/data/error/tung.rds"
input = "/ps/data/storage/leijiang/projects/Test-dataset_1710734872448/Results_normalization.h5ad"
adata = load_anndata(input)
adata

/ps/data/storage/leijiang/projects/Test-dataset_1710734872448/Results_normalization.h5ad


AnnData object with n_obs × n_vars = 2500 × 23341
    obs: 'nCount_SCT', 'nFeature_SCT', 'nCount_logCP10K', 'nFeature_logCP10K', 'nCount_logCPM', 'nFeature_logCPM', 'orig.ident', 'n_counts', 'n_genes', 'channel', 'tissue', 'subtissue', 'mouse.sex', 'mouse.id', 'percent.ercc', 'percent.ribo', 'free_annotation', 'cell_ontology_class', 'res.0.4', 'previous_free_annotation', 'previous_cell_ontology_class', 'cluster.ids', 'cell_ontology_id', 'pct_counts_mt'
    var: 'name'
    uns: 'pearson_residuals_normalization'
    layers: 'Pearson_residuals'

In [ ]:
adata.obsm.keys()

In [ ]:
sc.experimental.pp.normalize_pearson_residuals(adata)
adata
analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(adata, inplace=False)
analytic_pearson["X"]

In [ ]:
np.sum(analytic_pearson["X"])

In [ ]:
def _safe_accumulator_op(op, x, *args, **kwargs):
    """
    This function provides numpy accumulator functions with a float64 dtype
    when used on a floating point input. This prevents accumulator overflow on
    smaller floating point dtypes.
    Parameters
    ----------
    op : function
        A numpy accumulator function such as np.mean or np.sum.
    x : ndarray
        A numpy array to apply the accumulator function.
    *args : positional arguments
        Positional arguments passed to the accumulator function after the
        input x.
    **kwargs : keyword arguments
        Keyword arguments passed to the accumulator function.
    Returns
    -------
    result
        The output of the accumulator function passed to this function.
    """
    if np.issubdtype(x.dtype, np.floating) and x.dtype.itemsize < 8:
        result = op(x, *args, **kwargs, dtype=np.float64)
    else:
        result = op(x, *args, **kwargs)
    return result
    
print(np.isfinite(_safe_accumulator_op(np.sum, adata.layers["Pearson_residuals"])))

In [ ]:
pd.DataFrame.sparse.from_spmatrix(adata.X)

In [ ]:
sc.experimental.pp.normalize_pearson_residuals(adata)
adata

In [ ]:
adata.uns['pearson_residuals_normalization']

In [ ]:
import os

input = "/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
os.path.getsize(input)/(1024*1024)
os.path.splitext(input)[-1]

In [ ]:
methods = ['logCP10K', 'logCPM', 'SCT', 'scran', 'sctransform', 'sctransform_v2', 'DESeq2', 'RLE', 'TMM', 'upperquartile', 'UQ', 'TPM', 'FPKM', 'Pearson_residuals']
methods_to_remove = ['logCP10K', 'logCPM', 'SCT', 'scran', 'sctransform', 'sctransform_v2', 'DESeq2']
print(f"Methods to remove: {methods_to_remove}.")
if len(methods_to_remove) > 1:
    for method in methods_to_remove:
        methods.remove(method) # Remove method from methods list
print(f"Remaining methods: {methods}.")

In [ ]:
import os

output = "/usr/src/app/storage/leijiang/projects/Test-dataset_1710815922398/Results"
dataset = "Dataset"
process_id = 'aaa'

def get_output_path(path, process_id='', dataset=None, method = '', format = "AnnData"):
    output = os.path.abspath(path)
    method = '_' + method if method else ''
    directory, base_name = os.path.split(output.rstrip('/'))
    output_path = None

    if not os.path.exists(output):
        os.makedirs(output)

    if os.path.isdir(output):
        if dataset is None:
            dataset = base_name
        if format == "AnnData":
            output_path = os.path.join(output, process_id, dataset + method + ".h5ad")
            print("The output path is a directory, adding output file " + dataset + method + ".h5ad to the path.")
        elif format == "SingleCellExperiment":
            output_path = os.path.join(output, process_id, dataset + method + ".rds")
            print("The output path is a directory, adding output file " + dataset + method + ".rds to the path.")
        elif format == "Seurat":
            output_path = os.path.join(output, process_id, dataset + method + ".h5seurat")
            print("The output path is a directory, adding output file " + dataset + method + ".h5seurat to the path.")
        elif format == "CSV":
            output_path = os.path.join(output, process_id, dataset + method + ".csv")
            print("The output path is a directory, adding output file " + dataset + method + ".csv to the path.")
    else:
        if format == "AnnData":
            output_path = os.path.join(directory, process_id, base_name.replace(os.path.splitext(output)[-1], method + ".h5ad"))
        elif format == "SingleCellExperiment":
            output_path = os.path.join(directory, process_id, base_name.replace(os.path.splitext(output)[-1], method + ".rds"))
        elif format == "Seurat":
            output_path = os.path.join(directory, process_id, base_name.replace(os.path.splitext(output)[-1], method + ".h5seurat"))
        elif format == "CSV":
            output_path = os.path.join(directory, process_id, base_name.replace(os.path.splitext(output)[-1], method + ".csv"))
    
    return output_path

In [ ]:
output = get_output_path(dataset, output, method='normalization', format='Seurat')
output

In [ ]:
adata_path = get_output_path(dataset, output, method='normalization', format='AnnData')
adata_path

In [ ]:
directory, base_name = os.path.split(output.rstrip('/'))
output_path = os.path.join(directory, process_id, base_name.replace(os.path.splitext(output)[-1], ".h5ad"))
output_path

In [ ]:
adata.obs

In [ ]:
adata.obs.index.get_loc('TCGACGGGTACGAGCA-1-HCAHeart8287128')

In [ ]:
type(adata.X)

In [ ]:
adata.X.toarray()

In [ ]:
adata.layers['raw_counts'].toarray()

In [ ]:
adata.raw.X.toarray()

In [ ]:
"raw_counts" in adata.layers.keys()

In [ ]:
isinstance(adata.X, csr_matrix)

In [ ]:
np.set_printoptions(threshold=np.inf)
adata.X.toarray()[5,:]

In [ ]:
type(adata.X.toarray())

In [ ]:
isinstance(adata.X.toarray(), np.ndarray)

In [ ]:
np.max(adata.X)

In [ ]:
np.min(adata.X)

In [ ]:
from scipy.sparse import csr_matrix, find
find(adata.X)

In [ ]:
from scipy import stats
stats.describe(adata.X.toarray())

In [ ]:
np.median(adata.X.toarray())

In [ ]:
min_genes=200
min_cells=3
target_sum=1e4
n_neighbors=10
n_pcs=40
max_genes=0

In [ ]:
sc.pp.filter_cells(adata, min_genes=min_genes)
if max_genes is not None and max_genes != 0:
    sc.pp.filter_cells(adata, max_genes=max_genes)
sc.pp.filter_genes(adata, min_cells=min_cells)

In [ ]:
adata.var['mt']=adata.var_names.str.startswith('MT-')
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))

In [ ]:
adata.var

In [ ]:
adata.obs

In [ ]:
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True)

In [ ]:
adata.var_names_make_unique() 

# Filtering low quality reads
sc.pp.filter_cells(adata, min_genes=min_genes)
sc.pp.filter_genes(adata, min_cells=min_cells)
# mitochondrial genes
adata.var['mt']=adata.var_names.str.startswith('MT-')
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True)

adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)
adata.obs.outlier.value_counts()

adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 3) | (
    adata.obs["pct_counts_mt"] > 8
)
adata.obs.mt_outlier.value_counts()

print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")
adata.X

In [ ]:
# adata=adata[adata.obs.n_genes_by_counts < 2500, :]
# adata=adata[adata.obs.pct_counts_mt < 5, :]
import scrublet as scr
scrub = scr.Scrublet(adata.X, expected_doublet_rate = 0.076)
adata.obs['doublet_scores'], adata.obs['predicted_doublets'] = scrub.scrub_doublets(min_counts=2, min_cells=3, 
                                                        min_gene_variability_pctl=85, n_prin_comps=30)
adata.obs['predicted_doublets'].value_counts()
# adata=adata[adata.obs.predicted_doublets=="False", :]

sc.pp.normalize_total(adata, target_sum=target_sum)

sc.pp.log1p(adata)

In [ ]:
adata.X

In [ ]:
sc.pp.highly_variable_genes(adata)

In [ ]:
import scrublet as scr
scrub = scr.Scrublet(adata.X, expected_doublet_rate = 0.076)
adata.obs['doublet_scores'], adata.obs['predicted_doublets'] = scrub.scrub_doublets(min_counts=2, min_cells=3, 
                                                        min_gene_variability_pctl=85, n_prin_comps=30)
adata.obs['predicted_doublets'].value_counts()

In [ ]:
adata=adata[adata.obs.predicted_doublets==False, :]
adata

In [ ]:
adata = run_scanpy_qc(adata)
adata

In [ ]:
adata.X

In [ ]:
isinstance(adata.X, np.ndarray)

In [ ]:
adata = run_dimension_reduction(adata, layer="log10k")
adata

In [ ]:
adata.layers["raw_counts"].toarray()

In [ ]:
adata.X.toarray()

In [ ]:
adata.layers["log10k"] = adata.X

In [ ]:
del adata.layers["X"]
adata

In [ ]:
adata.uns

In [ ]:
adata.obs

In [ ]:
adata.obsm['log10k_umap'].shape

In [ ]:
adata.raw.X.toarray()

In [ ]:
adata.var.index

In [ ]:
type(adata.obs['total_counts'].tolist())

In [ ]:
adata.obs['n_counts']

In [ ]:
'pct_counts_mt' in adata.obs.columns

In [ ]:
adata.obs_vector('pct_counts_mt')

In [ ]:
train, test = sc_train_test_split(adata, train_fraction=0.8)

In [ ]:
train, val, test = sc_train_val_test_split(adata, train_fraction=0.8, validation_fraction=0.1, test_fraction=0.1)

In [ ]:
train

In [ ]:
val

In [ ]:
test

In [ ]:
test.X.toarray()

In [ ]:
test.layers["raw_counts"].toarray()

In [ ]:
test.write_h5ad(filename="/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin_test.h5ad", compression='gzip')

In [ ]:
adata = run_dimension_reduction(adata)

In [ ]:
# from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import normalized_mutual_info_score as NMI
from sklearn.metrics import silhouette_score
# from sklearn.mixture import GaussianMixture as GMM

def clustering_scores(labels, labels_pred, embedding):
    asw_score = silhouette_score(embedding, labels)
    nmi_score = NMI(labels, labels_pred)
    ari_score = ARI(labels, labels_pred)
    print(
        "Clustering Scores:\nSilhouette: %.4f\nNMI: %.4f\nARI: %.4f"
        % (asw_score, nmi_score, ari_score)
    )
    return asw_score, nmi_score, ari_score

In [ ]:
asw_score, nmi_score, ari_score = clustering_scores(adata.obs["cell_ontology_class"], adata.obs["leiden"], adata.obsp['connectivities'])

In [ ]:
adata.obsp['connectivities'].toarray()

In [ ]:
test.layers["log10k"].shape

In [ ]:
from scipy.sparse import csr_matrix
test.X = csr_matrix(test.X)
test.X

In [ ]:
test.layers["raw_counts"].toarray().shape

In [ ]:
adata.var

In [ ]:
list(adata.layers.keys())

In [ ]:
adata.n_vars

In [ ]:
adata.n_obs

In [ ]:
adata.obs_names.to_list()

In [ ]:
adata.var_names.to_list()

In [ ]:
list(adata.obsm.keys())

In [ ]:
adata.X

In [ ]:
type(adata.obs_keys())

In [ ]:
adata.var_keys()

In [ ]:
random_state=0
fraction=0.8
np.random.seed(random_state)
old_n_obs = adata.n_obs
old_n_obs
new_n_obs = int(fraction * old_n_obs)
obs_indices = np.random.choice(old_n_obs, size=new_n_obs, replace=False)
obs_indices

In [ ]:
test=adata[obs_indices].copy()
test

In [ ]:
test.var

In [ ]:
original_indices=np.arange(old_n_obs)
original_indices

In [ ]:
diff=np.setdiff1d(original_indices, obs_indices)
diff

In [ ]:
len(diff)

In [ ]:
np.random.shuffle(diff)
diff

In [ ]:
train=adata[diff].copy()
train

In [ ]:
train.obs